In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# To install fairseq from source and develop locally:
!git clone https://github.com/choyi0521/fairseq
%cd /content/fairseq
!pip install --editable .

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Cloning into 'fairseq'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12141 (delta 12), reused 16 (delta 8), pack-reused 12121
Receiving objects: 100% (12141/12141), 5.57 MiB | 4.25 MiB/s, done.
Resolving deltas: 100% (9056/9056), done.
/content/fairseq
Obtaining file:///content/fairseq
     |████████████████████████████████| 645kB 2.8MB/s 
  Running setup.

In [0]:
!git checkout dev3
!git pull origin dev3

Already on 'dev3'
Your branch is up to date with 'origin/dev3'.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/choyi0521/fairseq
 * branch            dev3       -> FETCH_HEAD
   2ef6d73..85f89f7  dev3       -> origin/dev3
Updating 2ef6d73..85f89f7
Fast-forward
 fairseq/modules/ddynamic_convolution.py | 12 ++++++------
 1 file changed, 6 insertions(+), 6 deletions(-)


In [0]:
# Download and prepare the data
%cd /content/fairseq
%cd examples/translation/
!bash prepare-iwslt14.sh
%cd ../..

# Preprocess/binarize the data
TEXT="examples/translation/iwslt14.tokenized.de-en"
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/iwslt14.tokenized.de-en \
    --workers 20

/content/fairseq
/content/fairseq/examples/translation
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 147458 (delta 101), reused 117 (delta 74), pack-reused 147295
Receiving objects: 100% (147458/147458), 129.72 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (113935/113935), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 530 (delta 9), reused 15 (delta 6), pack-reused 509
Receiving objects: 100% (530/530), 224.83 KiB | 612.00 KiB/s, done.
Resolving deltas: 100% (315/315), done.
--2019-11-18 14:26:41--  https://wit3.fbk.eu/archive/2014-01/texts/de/en/de-en.tgz
Resolving wit3.fbk.eu (wit3.fbk.eu)... 2

In [11]:

SAVE="/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt"

# Training
%cd /content/fairseq
!mkdir -p "{SAVE}" 
!CUDA_VISIBLE_DEVICES=0 $(which fairseq-train) data-bin/iwslt14.tokenized.de-en \
    --encoder-conv-type ddynamic --decoder-conv-type ddynamic \
    --clip-norm 0 --optimizer adam --lr 0.0005 \
    --source-lang de --target-lang en --max-tokens 4000 --no-progress-bar \
    --log-interval 100 --min-lr '1e-09' --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --lr-scheduler inverse_sqrt \
    --ddp-backend=no_c10d \
    --max-update 50000 --warmup-updates 4000 --warmup-init-lr '1e-07' \
    --adam-betas '(0.9, 0.98)' --keep-last-epochs 10 \
    -a lightconv_iwslt_de_en --save-dir "{SAVE}" \
    --dropout 0.3 --attention-dropout 0.1 --weight-dropout 0.1 \
    --encoder-glu 1 --decoder-glu 1 \
    --encoder-attention-heads 4 --decoder-attention-heads 4 \
    --encoder-attention-proj-heads 4 --decoder-attention-proj-heads 4 \
    --encoder-layers 7 --encoder-kernel-size-list [3,7,15,31,31,31,31] \
    2>&1 | tee "{SAVE}/train1.log"

/content/fairseq
Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='lightconv_iwslt_de_en', attention_dropout=0.1, best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data-bin/iwslt14.tokenized.de-en', dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=4, decoder_attention_proj_heads=4, decoder_conv_dim=512, decoder_conv_type='ddynamic', decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_glu=True, decoder_input_dim=512, decoder_kernel_size_list=[3, 7, 15, 31, 31, 31], decoder_layers=6, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.3, empty_cache_f

In [12]:
!python scripts/average_checkpoints.py --inputs "{SAVE}" \
    --num-epoch-checkpoints 10 --output "{SAVE}/checkpoint_last10_avg.pt"

# Evaluation
!CUDA_VISIBLE_DEVICES=0 fairseq-generate data-bin/iwslt14.tokenized.de-en --path "{SAVE}/checkpoint_last10_avg.pt" --batch-size 128 --beam 4 --remove-bpe --lenpen 1 --gen-subset test --quiet 

Namespace(checkpoint_upper_bound=None, inputs=['/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt'], num_epoch_checkpoints=10, num_update_checkpoints=None, output='/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint_last10_avg.pt')
averaging checkpoints:  ['/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint44.pt', '/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint43.pt', '/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint42.pt', '/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint41.pt', '/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint40.pt', '/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint39.pt', '/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint38.pt', '/content/gdrive/My Drive/colab/payless/save_dev3/dynamic_conv_iwslt/checkpoint37.pt', '/